In [ ]:
import uuid
from airflow import DAG
from airflow.operators.dummy import DummyOperator
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago


# task to training a model
def train_model(**context):
    # model training start
    # [model is training ...]
    # model training finish
    model_id = str(uuid.uuid4())
    # push key value via xcoms
    context["task_instance"].xcom_push(key="model_id", value=model_id)


# task to deploy model for serving
def deploy_model(**context):
    # get key value via xcoms
    model_id = context["task_instance"].xcom_pull(
        task_ids="train_model", key="model_id"
    )
    print(f"Deploying model {model_id}")


default_args = {
    'owner':'EMPLOYEE_ID', # owner是DAG的開發者, 例如: 員工8703147
}

with DAG(
        dag_id="deXX_04_xcoms",  # prefix必需是tenant id, 例如: de00
        start_date=days_ago(2),
        schedule_interval=None,
        default_args=default_args,
        access_control={
            'deXX': {'can_read', 'can_edit'}  # 設定DAG歸屬那個團隊[tenant id]與權限
        },
        tags=['de09'],
) as dag:
    start = DummyOperator(task_id="start")
    fetch_sales = DummyOperator(task_id="fetch_sales")
    clean_sales = DummyOperator(task_id="clean_sales")
    fetch_weather = DummyOperator(task_id="fetch_weather")
    clean_weather = DummyOperator(task_id="clean_weather")
    join_datasets = DummyOperator(task_id="join_datasets")
    train_model = PythonOperator(task_id="train_model", python_callable=train_model)
    deploy_model = PythonOperator(task_id="deploy_model", python_callable=deploy_model)

    start >> [fetch_sales, fetch_weather]
    fetch_sales >> clean_sales
    fetch_weather >> clean_weather
    [clean_sales, clean_weather] >> join_datasets
    join_datasets >> train_model >> deploy_model